In [3]:
import pandas as pd
import os
import random
import seaborn as sns
import matplotlib.pyplot as plt
import time
from datetime import datetime

In [51]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data.csv')
df = pd.read_csv(csv_file_path) 
df.sort_values(by=['userID','Timestamp'], inplace=True)
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


### Timestamp

In [52]:
def convert_time2timestamp(t):
    timestamp = time.mktime(t.timetuple())
    return int(timestamp)

def convert_string2datetime(s: str):
    return datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

def timestamp_fe(df):
    ## 문자열로 인식되는 Timestamp의 타입을 datetime으로 변경하기. 
    df["Timestamp"] = df["Timestamp"].apply(convert_string2datetime) # string type to datetime type


    ## 기본적인 시간정보 추가
    df['year'] = df['Timestamp'].dt.year
    df['month'] = df['Timestamp'].dt.month
    df['day'] = df['Timestamp'].dt.day
    df['hour'] = df['Timestamp'].dt.hour                                       # 시간대로 범주 추가 가능


    ## 요일 추가
    df['wday'] = df['Timestamp'].dt.weekday # Monday ~ Sunday => 0 ~ 6         # 주말로 범주 추가 가능
    

    ## 문제를 다시 풀어본 횟수 feature 'retry' 추가
    test_group = df.groupby(['userID','testId']) # 같은 시험지끼리 묶어준다.
    # retry_check = 0
    retry_df = pd.DataFrame()
    for key, group in test_group:
        if len(group[group.assessmentItemID == group.assessmentItemID.iloc[0]]) >= 2:
            retry_df = pd.concat([retry_df,group.groupby('assessmentItemID').cumcount()])
            # retry_check += 1
    retry_df.columns=['retry']
    df = pd.merge(df, retry_df, left_index=True,right_index=True, how="left")
    df['retry'] = df['retry'].fillna(0) # retry의 결측치(한 번만 푼 문제들)을 0으로 바꿔준다.

    
    ## 문제 풀이 시간 추가
    df['time'] = df['time'] = df.groupby(['userID','testId','retry'])['Timestamp'].diff().shift(-1) # 문제 풀이 시간
    df['time'] = df['time'].fillna(df['time'].median())                        # Null값은 중앙값으로 채우기.
    df['time'] = df['time'].apply(lambda x:x.total_seconds())                  # 년,월,일,날짜로 되어있는 값을 시간초로 변환
    df['time'] = df['time'].apply(lambda x:300 if x > 300 else x)              # 최댓값을 300으로 변환.


    ## 문제 풀이 시간 그룹 추가
    time_ranges = [-0.001,5,18,27,37,80,300]
    df['time_class'] = pd.cut(df['time'],time_ranges,labels=[0,1,2,3,4,5])
    
    ## 문제 풀이 시간 그룹별 통계량 추가
    time_class_stat = df[df['userID'].shift(1) == df['userID']].groupby(['time_class'])['answerCode'].agg(['mean','sum','std'])
    time_class_stat.columns = ['time_class_mean', 'time_class_sum', 'time_class_std']
    df = pd.merge(df,time_class_stat,on=['time_class'],how='left')


    ## 전체적인 시간정보를 나타내는 Timestamp는 int형으로 변환.
    df["Timestamp"] = df["Timestamp"].apply(convert_time2timestamp) # datetime to timestamp

    return df

In [53]:
df = timestamp_fe(df)

In [61]:
train_df = df[df['userID'].shift(1) != df['userID']]
train_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,wday,retry,time,time_class,time_class_mean,time_class_sum,time_class_std
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,1,0.0,3.0,0,0.272307,90691,0.445148
745,1,A040013001,A040000013,1,1578300043,2048,2020,1,6,8,0,0.0,183.0,5,0.761652,291686,0.426074
1678,2,A030050001,A030000050,1,1578654173,407,2020,1,10,11,4,0.0,52.0,4,0.779110,291229,0.414847
1954,5,A080001001,A080000001,1,1581537620,4604,2020,2,12,20,2,0.0,9.0,1,0.709332,280011,0.454071
2787,6,A030016001,A030000016,1,1583750908,7307,2020,3,9,10,0,0.0,20.0,2,0.644183,383374,0.478760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266515,7436,A050095001,A050000095,1,1583217245,360,2020,3,3,6,1,0.0,22.0,2,0.644183,383374,0.478760
2266530,7437,A040072001,A040000072,1,1585035915,2076,2020,3,24,7,1,0.0,32.0,3,0.784754,141408,0.410994
2266546,7438,A080002001,A080000002,1,1582664394,4605,2020,2,25,20,1,0.0,29.0,3,0.784754,141408,0.410994
2266562,7440,A050096001,A050000096,1,1597813039,5267,2020,8,19,4,2,0.0,45.0,4,0.779110,291229,0.414847


In [57]:
tdf = pd.read_csv(csv_file_path) 
tdf.sort_values(by=['userID','Timestamp'], inplace=True)
tdf.loc[tdf['userID'].shift(1) != tdf['userID'],'answerCode'] = -1
tdf = timestamp_fe(tdf)

In [60]:
test_df = tdf[tdf['userID'].shift(1) != tdf['userID']]
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,wday,retry,time,time_class,time_class_mean,time_class_sum,time_class_std
0,0,A060001001,A060000001,-1,1585009031,7224,2020,3,24,0,1,0.0,3.0,0,0.272307,90691,0.445148
745,1,A040013001,A040000013,-1,1578300043,2048,2020,1,6,8,0,0.0,183.0,5,0.761652,291686,0.426074
1678,2,A030050001,A030000050,-1,1578654173,407,2020,1,10,11,4,0.0,52.0,4,0.779110,291229,0.414847
1954,5,A080001001,A080000001,-1,1581537620,4604,2020,2,12,20,2,0.0,9.0,1,0.709332,280011,0.454071
2787,6,A030016001,A030000016,-1,1583750908,7307,2020,3,9,10,0,0.0,20.0,2,0.644183,383374,0.478760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266515,7436,A050095001,A050000095,-1,1583217245,360,2020,3,3,6,1,0.0,22.0,2,0.644183,383374,0.478760
2266530,7437,A040072001,A040000072,-1,1585035915,2076,2020,3,24,7,1,0.0,32.0,3,0.784754,141408,0.410994
2266546,7438,A080002001,A080000002,-1,1582664394,4605,2020,2,25,20,1,0.0,29.0,3,0.784754,141408,0.410994
2266562,7440,A050096001,A050000096,-1,1597813039,5267,2020,8,19,4,2,0.0,45.0,4,0.779110,291229,0.414847


In [62]:
diff =pd.concat([train_df,test_df],axis=1)
diff = diff.T.drop_duplicates(keep=False).T
diff = diff.drop(['answerCode'],axis=1)
diff

""
0
745
1678
1954
2787
...
2266515
2266530
2266546
2266562


In [63]:
df.to_csv('/opt/ml/input/data/train_data_timestamp.csv',index=False)